In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn import metrics

from sklearn.linear_model import Lasso
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB


import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
datA = pd.read_csv("datAClean.csv")
datB = pd.read_csv("datBClean.csv")
datC = pd.read_csv("datCClean.csv")
datD = pd.read_csv("datDClean.csv")
datE = pd.read_csv("datEClean.csv")
datF = pd.read_csv("datFClean.csv")
datG = pd.read_csv("datGClean.csv")
datH = pd.read_csv("datHClean.csv")
datI = pd.read_csv("datIClean.csv")
datJ = pd.read_csv("datJClean.csv")

dats = [datA, datB, datC, datD, datE, datF, datG, datH, datI, datJ]
dat = pd.concat(dats)

dat = dat.drop("Unnamed: 0", axis=1)

In [ ]:
dat.head()

In [ ]:
dat = dat.dropna()

In [ ]:
yColumns = ['Date & Time', 'onLineStatus', 'okStatus', 'runUpStatus', 'repairStatus', 'maintenanceStatus', 'faultStatus', 'gridFaultStatus', 'weatherOutageStatus', 'externalOutageStatus']

X = dat.drop(columns = yColumns,axis=1)
Y = dat['onLineStatus']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.33, random_state=45)

In [ ]:
Y.sum()/len(Y)

Training models to determine

In [ ]:
def trainlasso(x_train, x_test, y_train, y_test):
    lasso_regressor = Lasso(alpha = 0.1, fit_intercept= True, max_iter = 20000).fit(x_train, y_train)

    print("Training score (R^2): ", lasso_regressor.score(x_train, y_train))

    lasso_pred = lasso_regressor.predict(x_test)

    print("Testing score (R^2) : ", r2_score(y_test, lasso_pred))
    print("Mean Squared Error  : ", metrics.mean_squared_error(y_test, lasso_pred))


In [ ]:
def trainAll(x_train, x_test, y_train, y_test):
    #svm
    svm_model = svm.SVC(kernel='linear')
    svm_model.fit( x_train, y_train )
    #random forest
    rand_model = RandomForestClassifier(n_estimators=500)
    rand_model.fit( x_train, y_train )
    #naive bayes
    naive_model = GaussianNB()
    naive_model.fit( x_train, y_train )
    #decision tree
    dec_model = DecisionTreeClassifier()
    dec_model.fit( x_train, y_train )
    #log
    log_model = LogisticRegression()    
    log_model.fit( x_train, y_train )
    #lasso
    lasso_regressor = Lasso(alpha = 0.1, fit_intercept= True, max_iter = 20000).fit(x_train, y_train)




    log_pred = log_model.predict( x_test )          
    dec_pred = dec_model.predict( x_test )
    naive_pred = naive_model.predict( x_test )
    rand_pred = rand_model.predict( x_test )
    svm_pred = svm_model.predict( x_test )
    lasso_pred = lasso_regressor.predict(x_test)




    print("Accuracy on test set by logistic regression sklearn model   :  ", metrics.accuracy_score(y_test, log_pred) * 100)
    print("Accuracy on test set by decision tree sklearn model   :  ", metrics.accuracy_score(y_test, dec_pred) * 100)
    print("Accuracy on test set by naive bayes sklearn model   :  ", metrics.accuracy_score(y_test, naive_pred) * 100)
    print("Accuracy on test set by random forest sklearn model   :  ", metrics.accuracy_score(y_test, rand_pred) * 100)
    print("Accuracy on test set by support vector machine sklearn model   :  ", metrics.accuracy_score(y_test, svm_pred) * 100)
    print("Accuracy on test set by lasso regressor sklearn model   :  ", metrics.accuracy_score(y_test, lasso_pred) * 100)

In [ ]:
trainAll(x_train, x_test, y_train, y_test)
trainlasso(x_train, x_test, y_train, y_test)

Recursive Feature Elimination

In [ ]:
# importing required libraries
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

# create the object of the model
lreg = LogisticRegression()


# specify the number of  features to select 
rfe = RFE(lreg, n_features_to_select=5)

# fit the model
rfe = rfe.fit(x_train, y_train)

In [ ]:
print('\n\nFEATUERS SELECTED\n\n')
print(rfe.support_)

print('\n\nRANKING OF FEATURES\n\n')
print(rfe.ranking_)

y_pred = rfe.predict(x_test)
print("Testing score (R^2) : ", metrics.accuracy_score(y_test, y_pred) * 100)

In [ ]:
X[X.columns[rfe.get_support()]]

performance for each status value

In [ ]:
def allYs():
  arrs = []
  yColumns = ['Date & Time', 'onLineStatus', 'okStatus', 'runUpStatus', 'repairStatus', 'maintenanceStatus', 'faultStatus', 'gridFaultStatus', 'weatherOutageStatus', 'externalOutageStatus']
  for y in yColumns:
      if y != 'Date & Time':

        X = dat.drop(columns = [x for x in yColumns if x != y],axis=1)
        Y = dat['onLineStatus']

        x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.33, random_state=45)

        log_model = LogisticRegression()    
        log_model.fit( x_train, y_train )

        log_pred = log_model.predict( x_test )          

        arrs += [" ".join(["Accuracy on test set ", str(y),   " by logistic regression sklearn model   :  ", str(metrics.accuracy_score(y_test, log_pred) * 100)])] 
  return(arrs)

In [ ]:
strs = allYs()

In [ ]:
strs

energy lost when out

In [ ]:
dat[dat['onLineStatus'] != 1][['EXPECTED POWER', 'POWER']]

In [ ]:
dat[dat['onLineStatus'] != 1][['EXPECTED POWER']].mean().iloc[0] - (dat[dat['onLineStatus'] != 1][['POWER']].mean()).iloc[0]

Identify faulty turbines

In [ ]:
def turbinePred (dats, log_model):
  arrs = []
  for d in dats:
    d = d.drop(["Unnamed: 0", 'GEARBOX BEARING TEMPERATURE INTERMEDIATE SHAFT GENERATOR SIDE (ROTOR SIDE)'], axis=1, errors='ignore')
    d = d.dropna()
    yColumns = ['Date & Time', 'onLineStatus', 'okStatus', 'runUpStatus', 'repairStatus', 'maintenanceStatus', 'faultStatus', 'gridFaultStatus', 'weatherOutageStatus', 'externalOutageStatus']

    X = d.drop(columns = yColumns,axis=1) 
    Y = d['onLineStatus']

    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.33, random_state=45)  

    log_pred = log_model.predict( x_test )          
    arrs += [log_pred.sum() / len(log_pred)] 

  return(arrs)

In [ ]:
log_model = trainAll(x_train, x_test, y_train, y_test)
arrs = turbinePred(dats, log_model)

In [ ]:
arrs